### UBAH EXCEL KE CSV

In [4]:
import pandas as pd

# --- PENGATURAN ---
# Ganti dengan nama file Excel Anda
nama_file_xlsx = r'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data.xlsx'

# Ganti dengan nama file CSV yang Anda inginkan sebagai output
nama_file_csv = r'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data2.csv'
# --------------------

try:
    # Langkah 1: Baca file Excel, definisikan bahwa 4 baris pertama adalah header
    # header=[0, 1, 2, 3] memberitahu pandas untuk membaca 4 baris teratas sebagai header.
    print(f"Membaca file '{nama_file_xlsx}' dengan header multi-baris...")
    df = pd.read_excel(nama_file_xlsx, header=[0, 1, 2, 3])

    # Langkah 2: Membersihkan dan menggabungkan nama kolom dari multi-baris
    # Loop ini akan mengambil setiap level header dan menggabungkannya dengan '_'
    print("Membersihkan dan menggabungkan nama kolom...")
    new_columns = []
    for col in df.columns:
        # Mengambil setiap bagian dari nama kolom, kecuali yang berisi 'Unnamed'
        parts = [str(part) for part in col if 'Unnamed' not in str(part)]
        # Menggabungkannya dengan underscore '_'
        new_columns.append('_'.join(parts).strip())
    
    # Menetapkan nama kolom yang sudah bersih ke DataFrame
    df.columns = new_columns

    # Langkah 3: Simpan data ke dalam file .csv
    print(f"Mengonversi ke '{nama_file_csv}'...")
    df.to_csv(nama_file_csv, index=False)
    
    print("\nKonversi berhasil! 🎉")
    print(f"File '{nama_file_csv}' telah dibuat dengan format yang benar.")

except FileNotFoundError:
    print(f"\nERROR: File '{nama_file_xlsx}' tidak ditemukan.")
    print("Pastikan nama file sudah benar dan file tersebut berada di folder yang sama dengan skrip ini.")
except Exception as e:
    print(f"\nTerjadi sebuah error: {e}")

Membaca file 'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data.xlsx' dengan header multi-baris...
Membersihkan dan menggabungkan nama kolom...
Mengonversi ke 'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data2.csv'...

Konversi berhasil! 🎉
File 'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data2.csv' telah dibuat dengan format yang benar.


### CLEANING KOLOM BERTH LOCATION

In [6]:
import pandas as pd
import re

# --- SETUP PATH ---
csv_path = r'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data.csv'
csv_out = r'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data_Cleaned2.csv'

# --- 1. Baca CSV ---
df = pd.read_csv(csv_path)

# --- 2. Print semua kolom ---
print("Daftar kolom dalam dataset:")
print(list(df.columns))

# --- 3. Cleaning kolom BERTH LOCATION ---
def clean_port(value):
    if pd.isna(value):
        return None
    # Uppercase
    val = str(value).upper().strip()
    # Hilangkan angka di akhir / tengah (misal CELEBES107 -> CELEBES, MSA 210 -> MSA)
    val = re.sub(r'\d+', '', val).strip()
    return val

df['BERTH LOCATION'] = df['BERTH LOCATION'].apply(clean_port)

# --- 4. Ringkasan unique value setelah cleaning ---
unique_clean = df['BERTH LOCATION'].dropna().unique()
unique_count_clean = df['BERTH LOCATION'].nunique()

print("\n--- Ringkasan BERTH LOCATION setelah cleaning ---")
print(f"Jumlah unique value : {unique_count_clean}")
print("Daftar unique value :", unique_clean)
print(df['BERTH LOCATION'].value_counts())

# --- 5. Simpan hasil update ke CSV baru ---
df.to_csv(csv_out, index=False)
print(f"\n✅ File cleaned berhasil disimpan ke: {csv_out}")

Daftar kolom dalam dataset:
['VESSEL ID (DMY)', 'Voyage No.', 'Voyage Yr', 'BERTH LOCATION', 'Date Berth', 'TOTAL BONGKARAN_EMPTY_20 DC', 'TOTAL BONGKARAN_EMPTY_40 HC', 'TOTAL BONGKARAN_FULL_20 DC', 'TOTAL BONGKARAN_FULL_40 HC', 'PENGAJUAN KE PLANNER_EMPTY_20 DC', 'PENGAJUAN KE PLANNER_EMPTY_40 HC', 'PENGAJUAN KE PLANNER_FULL_20 DC', 'PENGAJUAN KE PLANNER_FULL_40 HC', 'ACC PENGAJUAN_EMPTY_20 DC', 'ACC PENGAJUAN_EMPTY_40 HC', 'ACC PENGAJUAN_FULL_20 DC', 'ACC PENGAJUAN_FULL_40 HC', '0_TOTAL_BOX_20 DC', '0_40 HC', '0_TEUS', 'REALISASI ALL DEPO_ALL DEPO_MXD_20 DC', 'REALISASI ALL DEPO_ALL DEPO_MXD_40 HC', 'REALISASI ALL DEPO_ALL DEPO_FXD_20 DC', 'REALISASI ALL DEPO_ALL DEPO_FXD_40 HC', 'SHIPSIDE_YES_MXD_20 DC', 'SHIPSIDE_YES_MXD_40 HC', 'SHIPSIDE_YES_FXD_20 DC', 'SHIPSIDE_YES_FXD_40 HC', 'SHIPSIDE_NO_MXD_20 DC', 'SHIPSIDE_NO_MXD_40 HC', 'SHIPSIDE_NO_FXD_20 DC', 'SHIPSIDE_NO_FXD_40 HC', '0_TOTAL_BOX_20 DC.1', '0_40 HC.1', '0_TEUS.1', 'TURUN CY_BOX_20 DC', 'TURUN CY_BOX_40 HC', 'TURUN CY_TEU

### FORMATTING DATE BERTH

In [ ]:
import pandas as pd

def parse_flexible_date(date_str):
    """
    Fungsi untuk mengubah string tanggal menjadi objek datetime.
    Secara eksplisit menangani format non-standar seperti 'dd/mm/yyyy HH.MM'
    sebelum beralih ke deteksi otomatis.
    """
    if pd.isna(date_str):
        return pd.NaT # Kembalikan 'Not a Time' jika datanya kosong
    
    # Coba format non-standar terlebih dahulu (misal: 08/05/2024 18.00)
    try:
        return pd.to_datetime(date_str, format='%d/%m/%Y %H.%M')
    except (ValueError, TypeError):
        # Jika format di atas gagal, coba deteksi otomatis untuk format lain
        try:
            # dayfirst=True membantu pandas menginterpretasikan dd/mm/yyyy dengan benar
            return pd.to_datetime(date_str, dayfirst=True, infer_datetime_format=True)
        except (ValueError, TypeError):
            # Jika semua format gagal, kembalikan NaT
            return pd.NaT

# 1. Tentukan path file Anda
# Pastikan untuk mengganti path ini jika lokasi file Anda berbeda.
input_csv_path = r'E:\$7th\PTSPIL\Ship-Operation-Systems\EDA-Model\Ship_Operation_Data_Cleaned.csv'
output_csv_path = r'E:\$7th\PTSPIL\Ship-Operation-Systems\project\data\Ship_Operation_Data_Cleaned.csv'

try:
    # 2. Baca file CSV ke dalam DataFrame pandas
    df = pd.read_csv(input_csv_path)

    print("✅ File CSV berhasil dibaca.")
    print("Contoh format 'Date Berth' sebelum diubah:")
    print(df['Date Berth'].head())
    print("-" * 50)

    # 3. Terapkan fungsi fleksibel untuk konversi tanggal
    # Ini akan menangani berbagai format yang ada di kolom Anda.
    original_dtype = df['Date Berth'].dtype
    if pd.api.types.is_string_dtype(original_dtype):
        df['Date Berth'] = df['Date Berth'].apply(parse_flexible_date)
    else:
        # Jika sudah bukan string, coba konversi langsung
        df['Date Berth'] = pd.to_datetime(df['Date Berth'], errors='coerce')
    
    # 4. Hapus baris di mana 'Date Berth' kosong (null) setelah konversi
    initial_rows = len(df)
    df.dropna(subset=['Date Berth'], inplace=True)
    dropped_rows = initial_rows - len(df)
    if dropped_rows > 0:
        print(f"✅ Sebanyak {dropped_rows} baris dengan data tanggal kosong/salah telah dihapus.")
    else:
        print("✅ Tidak ada baris dengan data tanggal kosong yang ditemukan.")
    print("-" * 50)


    # 5. Ubah kembali ke format string yang seragam (YYYY-MM-DD HH:MM:SS)
    # Ini adalah format standar yang mudah dibaca oleh Python dan database.
    df['Date Berth'] = df['Date Berth'].dt.strftime('%Y-%m-%d %H:%M:%S')

    print("✅ Kolom 'Date Berth' telah berhasil diseragamkan.")
    print("Contoh format 'Date Berth' setelah diubah:")
    print(df['Date Berth'].head())
    print("-" * 50)

    # 6. Simpan DataFrame yang sudah bersih ke file CSV baru
    df.to_csv(output_csv_path, index=False)

    print(f"✅ File baru yang sudah bersih telah disimpan di:\n{output_csv_path}")

except FileNotFoundError:
    print(f"❌ ERROR: File tidak ditemukan di path: {input_csv_path}")
    print("   Mohon periksa kembali path file Anda.")
except Exception as e:
    print(f"❌ Terjadi error: {e}")



✅ File CSV berhasil dibaca.
Contoh format 'Date Berth' sebelum diubah:
0    2024-01-03 08:00:00
1    2024-01-04 06:00:00
2    2024-01-04 12:00:00
3    2024-01-03 00:00:00
4    2024-01-04 05:00:00
Name: Date Berth, dtype: object
--------------------------------------------------


C:\Users\LEGION\AppData\Local\Temp\ipykernel_25732\2654990375.py:19: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(date_str, dayfirst=True, infer_datetime_format=True)
C:\Users\LEGION\AppData\Local\Temp\ipykernel_25732\2654990375.py:19: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(date_str, dayfirst=True, infer_datetime_format=True)


✅ Sebanyak 2 baris dengan data tanggal kosong/salah telah dihapus.
--------------------------------------------------
✅ Kolom 'Date Berth' telah berhasil diseragamkan.
Contoh format 'Date Berth' setelah diubah:
0    2024-03-01 08:00:00
1    2024-04-01 06:00:00
2    2024-04-01 12:00:00
3    2024-03-01 00:00:00
4    2024-04-01 05:00:00
Name: Date Berth, dtype: object
--------------------------------------------------
✅ File baru yang sudah bersih telah disimpan di:
E:\$7th\PTSPIL\Ship-Operation-Systems\project\data\Ship_Operation_Cleaned.csv
